### 문제 2-1 : 콤마 구분 리스트 파서 활용

In [2]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [14]:
import os
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_openai import ChatOpenAI

# 출력 파서 정의
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    input_variables=["subject"],
    template="""
사용자가 관심있는 분야는 '{subject}'입니다.
이 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분하여 추천해 주세요.

{format_instructions}
""",
    partial_variables={"format_instructions": format_instructions}
)

# Groq 기반 LLM 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 구성 (프롬프트 → 모델 → 파서)
chain: RunnableSequence = prompt | model | output_parser

# 테스트 실행
test_subjects = ["음식", "영화", "스포츠", "전통문화"]

print("=== 한국 추천 장소/활동 리스트 ===\n")

for subject in test_subjects:
    try:
        result = chain.invoke({"subject": subject})
        print(f"[{subject}] → {result}")
    except Exception as e:
        print(f"[{subject}] 처리 중 오류: {e}")


=== 한국 추천 장소/활동 리스트 ===

[음식] → ['서울의 N서울타워', '경주의 황남빵마을', '부산의 자갈치시장', '경기도의 양평시장', '전북의 전주한옥마을']
[영화] → ['부산국제영화제', '영화진흥위원회', '한국영화박물관', '충무로 영화 거리', 'CGV']
[스포츠] → ['서울올림픽공원', '수원월드컵경기장', '부산아시아드주경기장', '강릉올림픽파크', '제주월드컵경기장']
[전통문화] → ['종묘', '창덕궁', '국립무형유산원', '강릉 한옥마을', '전주 한벽루']


### 문제 2-2 : 영화 리뷰 감정 분석기


In [18]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain_core.runnables import RunnableSequence
from enum import Enum

# .env 파일 로드
load_dotenv()

# 감정 Enum 정의
class Sentiment(str, Enum):
    긍정 = "긍정"
    부정 = "부정"
    보통 = "보통"

# 파서 정의
enum_parser = EnumOutputParser(enum=Sentiment)

# LLM 설정 (env의 OPENAI_API_KEY 사용)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 파서 고도화 (출력 실패 시 수정)
fixing_parser = OutputFixingParser.from_llm(parser=enum_parser, llm=llm)

# 프롬프트 구성
prompt = PromptTemplate(
    input_variables=["review"],
    partial_variables={"format_instructions": enum_parser.get_format_instructions()},
    template="""
다음 영화 리뷰를 읽고 감정을 분류하세요.
감정은 반드시 다음 중 하나로만 답하세요: 긍정, 부정, 보통

리뷰: "{review}"

{format_instructions}
"""
)

# 체인 구성
chain: RunnableSequence = prompt | llm | fixing_parser

# 테스트
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

for r in test_reviews:
    result = chain.invoke({"review": r})
    print(f"리뷰: \"{r}\"\n→ 감정: {result}\n")


리뷰: "이 영화 정말 재미없어요. 시간 낭비였습니다."
→ 감정: Sentiment.부정

리뷰: "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!"
→ 감정: Sentiment.긍정

리뷰: "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
→ 감정: Sentiment.보통



### 문제 2-3: 학생 정보 구조화 시스템

In [23]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnableSequence
from pydantic import BaseModel, Field
from typing import List

# 환경 변수 로드
load_dotenv()

# Pydantic 모델 정의
class StudentInfo(BaseModel):
    name: str = Field(..., description="학생의 이름")
    age: int = Field(..., description="학생의 나이")
    major: str = Field(..., description="학생의 전공")
    hobbies: List[str] = Field(..., description="학생의 취미 리스트")
    goal: str = Field(..., description="학생의 목표")

# 출력 파서 생성
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 프롬프트 정의
prompt = PromptTemplate(
    input_variables=["intro"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
    template="""
다음 자기소개 문장을 읽고 이름(name), 나이(age), 전공(major), 취미(hobbies), 목표(goal)를 추출하세요.
{format_instructions}

자기소개:
"{intro}"
"""
)

# LLM (Groq API 사용)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 체인 구성
chain: RunnableSequence = prompt | llm | parser

# 테스트 입력
intro_text = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

# 실행
try:
    result = chain.invoke({"intro": intro_text})
    print("구조화된 학생 정보:")
    print(result.model_dump_json(indent=4))
except Exception as e:
    print(f"오류 발생: {e}")


구조화된 학생 정보:
{
    "name": "김민수",
    "age": 22,
    "major": "컴퓨터공학",
    "hobbies": [
        "게임하기",
        "영화보기",
        "코딩"
    ],
    "goal": "훌륭한 개발자가 되는 것"
}


### 문제 2-4 : 여행 계획 분석기

In [25]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.runnables import RunnableSequence

# 환경 변수 로드
load_dotenv()

# 파싱할 필드 정의 (ResponseSchema)
schemas = [
    ResponseSchema(name="destination", description="여행한 장소 (도시 또는 지역명)"),
    ResponseSchema(name="duration", description="여행 기간 (예: 2박 3일)"),
    ResponseSchema(name="budget", description="총 예산 또는 지출 (예: 30만원)"),
    ResponseSchema(name="rating", description="여행 만족도 (1~5점 중 하나)"),
    ResponseSchema(name="activities", description="주요 활동 리스트 (예: 관광, 먹거리 등)")
]

# StructuredOutputParser 생성
parser = StructuredOutputParser.from_response_schemas(schemas)

# 프롬프트 템플릿 정의
prompt = PromptTemplate(
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
    template="""
다음 여행 후기나 계획에서 핵심 정보를 추출하세요.
{format_instructions}

여행 글:
"{text}"
"""
)

# LLM 설정 (Groq 기반 LLaMA)
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 체인 구성
chain: RunnableSequence = prompt | llm | parser

# 테스트 입력
text = (
    "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 "
    "해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. "
    "정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
)

# 실행
try:
    result = chain.invoke({"text": text})
    print("구조화된 여행 계획:")
    import json
    print(json.dumps(result, indent=4, ensure_ascii=False))
except Exception as e:
    print(f"오류 발생: {e}")


구조화된 여행 계획:
{
    "destination": "부산",
    "duration": "2박3일",
    "budget": "30만원",
    "rating": "4점",
    "activities": "해운대 바다구경, 자갈치시장 회, 감천문화마을 구경"
}
